# Data preparation for Google Map test 
## --subset the data that will be used in distance matrix calculation--

In [1]:
#pip install -U googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.4.5-py3-none-any.whl size=37983 sha256=24a3079dae8a68e090962b9a7f40e5f44af9ae8aed32473f66f644801445e079
  Stored in directory: /home/jovyan/.cache/pip/wheels/aa/52/df/fcc993a8582e4b3953e7217163b01c9ebcc36fe08f4090ba7e
Successfully built googlemaps
Note: you may need to restart the kernel to use updated packages.


In [2]:
from access import access, weights, datasets
import logging
import pandas as pd
import geopandas as gpd

import googlemaps

## Get the centroid of MSOA (demand points)

In [22]:
MSOA_points=gpd.read_file('https://github.com/LingruFeng/dissertation/blob/main/data_github/England_basemap/MSOA_Population_Weighted_Centroids_with_population.gpkg?raw=true')
#MSOA_points = MSOA_points.to_crs('epsg:27700')
MSOA_points = MSOA_points.to_crs('epsg:4326')
MSOA_points['MSOA_lon'] = MSOA_points['geometry'].x
MSOA_points['MSOA_lat'] = MSOA_points['geometry'].y

In [23]:
MSOA_points

,objectid,msoa11cd,population,geometry,MSOA_lon,MSOA_lat
0,1,E02002536,9771,POINT (-1.29577 54.61069),-1.295773,54.610691
1,2,E02002537,8641,POINT (-1.27726 54.61131),-1.277263,54.611313
2,3,E02002534,5693,POINT (-1.05346 54.52765),-1.053458,54.527646
3,4,E02002535,9336,POINT (-1.28729 54.62215),-1.287295,54.622148
4,5,E02002532,6655,POINT (-1.05793 54.53718),-1.057931,54.537176
...,...,...,...,...,...,...
6786,6787,E02004669,7496,POINT (-2.09642 51.94596),-2.096424,51.945960
6787,6788,E02006096,7001,POINT (-2.95612 50.87855),-2.956122,50.878552
6788,6789,E02003088,8444,POINT (-2.81548 51.32678),-2.815479,51.326784
6789,6790,E02006070,8291,POINT (-2.97724 51.12711),-2.977239,51.127113


In [24]:
MSOA_points_df = MSOA_points[['msoa11cd','MSOA_lon','MSOA_lat']]
MSOA_points_df

,msoa11cd,MSOA_lon,MSOA_lat
0,E02002536,-1.295773,54.610691
1,E02002537,-1.277263,54.611313
2,E02002534,-1.053458,54.527646
3,E02002535,-1.287295,54.622148
4,E02002532,-1.057931,54.537176
...,...,...,...
6786,E02004669,-2.096424,51.945960
6787,E02006096,-2.956122,50.878552
6788,E02003088,-2.815479,51.326784
6789,E02006070,-2.977239,51.127113


## read vaccination site point data (supply points)

In [13]:
site = gpd.read_file('https://github.com/LingruFeng/dissertation/blob/main/data_github/site_2_26_clean/vaccination_site.gpkg?raw=true')
# site = site.to_crs('epsg:27700')
site = site.to_crs('epsg:4326')
site['site_lon'] = site['geometry'].x
site['site_lat'] = site['geometry'].y

In [17]:
site_df = site[['index','site_lon','site_lat']]
site_df

,index,site_lon,site_lat
0,0,-0.471211,52.128622
1,1,-0.474637,51.894793
2,2,-0.093771,51.468063
3,3,-0.093771,51.468063
4,4,-0.484367,51.891807
...,...,...,...
1595,1595,-4.865818,50.515949
1596,1596,-0.205835,51.433441
1597,1597,-0.173830,51.514861
1598,1598,-1.879170,50.716914


In [14]:
site

,index,supply_value,geometry,site_lon,site_lat
0,0,1,POINT (-0.47121 52.12862),-0.471211,52.128622
1,1,1,POINT (-0.47464 51.89479),-0.474637,51.894793
2,2,1,POINT (-0.09377 51.46806),-0.093771,51.468063
3,3,1,POINT (-0.09377 51.46806),-0.093771,51.468063
4,4,1,POINT (-0.48437 51.89181),-0.484367,51.891807
...,...,...,...,...,...
1595,1595,1,POINT (-4.86582 50.51595),-4.865818,50.515949
1596,1596,1,POINT (-0.20584 51.43344),-0.205835,51.433441
1597,1597,1,POINT (-0.17383 51.51486),-0.173830,51.514861
1598,1598,1,POINT (-1.87917 50.71691),-1.879170,50.716914


## Calculate the Euclidean distance from demand to supply locations 
## --threshold: 10 miles (16093.44 meters)--

In [15]:
# Load each of the example datasets which correspond to the demand (population) and supply ('hospital','pcn','pharmacy','vc') respectively.
distance = access(demand_df = MSOA_points,
           demand_index='msoa11cd',
           demand_value='population',
           supply_df= site,
           supply_index= 'index',
           supply_value=['supply_value']
                 )
# reproject
distance.demand_df = distance.demand_df.to_crs('epsg:27700')
distance.supply_df = distance.supply_df.to_crs('epsg:27700')

In [16]:
##### Take 16093.44 meters (10 miles) as the calculation threshold
distance.create_euclidean_distance(threshold = 16093.44)
distance.cost_df

,euclidean,origin,dest
0,4573.214894,E02002536,122
1,8449.407559,E02002536,127
2,8217.025264,E02002536,137
3,9944.296934,E02002536,836
4,6707.555358,E02002536,838
...,...,...,...
555153,12599.535851,E02003088,1204
555154,5650.422238,E02006070,1278
555155,14527.209707,E02006679,1176
555156,9895.394094,E02006679,1177


In [35]:
euclidean_distance_matrix = distance.cost_df

In [38]:
euclidean_distance_matrix = pd.merge(left = euclidean_distance_matrix, 
                                     right = site_df, 
                                     how = 'left', 
                                     left_on = 'dest', 
                                     right_on = 'index')

euclidean_distance_matrix = pd.merge(left = euclidean_distance_matrix, 
                                     right = MSOA_points_df, 
                                     how = 'left', 
                                     left_on = 'origin', 
                                     right_on = 'msoa11cd')

,euclidean,origin,dest,index,site_lon,site_lat,msoa11cd,MSOA_lon,MSOA_lat
0,4573.214894,E02002536,122,122,-1.347271,54.582497,E02002536,-1.295773,54.610691
1,8449.407559,E02002536,127,127,-1.214583,54.551181,E02002536,-1.295773,54.610691
2,8217.025264,E02002536,137,137,-1.217247,54.552620,E02002536,-1.295773,54.610691
3,9944.296934,E02002536,836,836,-1.213812,54.686367,E02002536,-1.295773,54.610691
4,6707.555358,E02002536,838,838,-1.216531,54.571757,E02002536,-1.295773,54.610691
...,...,...,...,...,...,...,...,...,...
555153,12599.535851,E02003088,1204,1204,-2.754862,51.433531,E02003088,-2.815479,51.326784
555154,5650.422238,E02006070,1278,1278,-3.015693,51.082446,E02006070,-2.977239,51.127113
555155,14527.209707,E02006679,1176,1176,-2.345549,51.360932,E02006679,-2.136956,51.364067
555156,9895.394094,E02006679,1177,1177,-1.996076,51.352366,E02006679,-2.136956,51.364067


In [39]:
euclidean_distance_matrix.drop(columns=['msoa11cd','index'], axis=1,inplace=True)
euclidean_distance_matrix

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
0,4573.214894,E02002536,122,-1.347271,54.582497,-1.295773,54.610691
1,8449.407559,E02002536,127,-1.214583,54.551181,-1.295773,54.610691
2,8217.025264,E02002536,137,-1.217247,54.552620,-1.295773,54.610691
3,9944.296934,E02002536,836,-1.213812,54.686367,-1.295773,54.610691
4,6707.555358,E02002536,838,-1.216531,54.571757,-1.295773,54.610691
...,...,...,...,...,...,...,...
555153,12599.535851,E02003088,1204,-2.754862,51.433531,-2.815479,51.326784
555154,5650.422238,E02006070,1278,-3.015693,51.082446,-2.977239,51.127113
555155,14527.209707,E02006679,1176,-2.345549,51.360932,-2.136956,51.364067
555156,9895.394094,E02006679,1177,-1.996076,51.352366,-2.136956,51.364067


In [40]:
distance.cost_df.to_csv('euclidean_distance.csv',index=False)

In [3]:
#euclidean_distance_matrix = pd.read_csv('euclidean_distance.csv')
#euclidean_distance_matrix

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
0,4573.214894,E02002536,122,-1.347271,54.582497,-1.295773,54.610691
1,8449.407559,E02002536,127,-1.214583,54.551181,-1.295773,54.610691
2,8217.025264,E02002536,137,-1.217247,54.552620,-1.295773,54.610691
3,9944.296934,E02002536,836,-1.213812,54.686367,-1.295773,54.610691
4,6707.555358,E02002536,838,-1.216531,54.571757,-1.295773,54.610691
...,...,...,...,...,...,...,...
555153,12599.535851,E02003088,1204,-2.754862,51.433531,-2.815479,51.326784
555154,5650.422238,E02006070,1278,-3.015693,51.082446,-2.977239,51.127113
555155,14527.209707,E02006679,1176,-2.345549,51.360932,-2.136956,51.364067
555156,9895.394094,E02006679,1177,-1.996076,51.352366,-2.136956,51.364067


In [52]:
euclidean_distance_matrix_sub=euclidean_distance_matrix[0:5]
euclidean_distance_matrix_sub

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
0,4573.214894,E02002536,122,-1.347271,54.582497,-1.295773,54.610691
1,8449.407559,E02002536,127,-1.214583,54.551181,-1.295773,54.610691
2,8217.025264,E02002536,137,-1.217247,54.552620,-1.295773,54.610691
3,9944.296934,E02002536,836,-1.213812,54.686367,-1.295773,54.610691
4,6707.555358,E02002536,838,-1.216531,54.571757,-1.295773,54.610691


# An API request 10,000 times, divided into 6 accounts to request all the distance

## sub 0 — — 100000 :

In [62]:
#euclidean_distance_matrix_sub0=euclidean_distance_matrix[0:1000]
#euclidean_distance_matrix_sub00=euclidean_distance_matrix[1000:2000]
#euclidean_distance_matrix_sub000=euclidean_distance_matrix[2000:10000]
#euclidean_distance_matrix_sub0000=euclidean_distance_matrix[10000:40005]
#euclidean_distance_matrix_sub00000=euclidean_distance_matrix[40005:100000]

In [63]:
euclidean_distance_matrix_sub00000

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
40005,9065.184528,E02006230,320,1.026239,51.977340,0.958806,52.047371
40006,699.587089,E02006230,328,0.951969,52.042708,0.958806,52.047371
40007,14250.236081,E02006230,382,0.752463,52.032980,0.958806,52.047371
40008,15882.798416,E02006230,1520,1.188752,52.031249,0.958806,52.047371
40009,3974.639134,E02006231,319,1.148655,52.064023,1.106164,52.039749
...,...,...,...,...,...,...,...
99995,5833.769441,E02000870,848,-0.132245,51.521891,-0.048735,51.527933
99996,4423.187249,E02000870,850,-0.082066,51.561831,-0.048735,51.527933
99997,9593.039885,E02000870,853,-0.185497,51.515375,-0.048735,51.527933
99998,7413.892529,E02000870,854,-0.140230,51.493547,-0.048735,51.527933


In [ ]:
print(euclidean_distance_matrix_sub0.shape,
      euclidean_distance_matrix_sub00.shape,
      euclidean_distance_matrix_sub000.shape,
      euclidean_distance_matrix_sub0000.shape，
      euclidean_distance_matrix_sub00000.shape)

### 0-1000：

In [ ]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub0.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    #result_time = result["rows"][0]["elements"][0]["duration"]["value"]
    #time_list.append(round(result_time/60,1))
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)
    
output0 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
#output['duration in minutes'] = time_list
output0['origin_id'] = origin_id_list
output0['destination_id'] = destination_id_list

### 1000-2000：

In [ ]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub00.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    #result_time = result["rows"][0]["elements"][0]["duration"]["value"]
    #time_list.append(round(result_time/60,1))
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)
    
output00 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
#output['duration in minutes'] = time_list
output00['origin_id'] = origin_id_list
output00['destination_id'] = destination_id_list

### 2000-10000：

In [108]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

for (i1, row1) in euclidean_distance_matrix_sub000.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    #result_time = result["rows"][0]["elements"][0]["duration"]["value"]
    #time_list.append(round(result_time/60,1))
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)
    
output000 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
#output['duration in minutes'] = time_list
output000['origin_id'] = origin_id_list
output000['destination_id'] = destination_id_list

In [111]:
output000

,Distance in meter,origin_id,destination_id
0,14431,E02001694,119
1,14431,E02001694,120
2,19915,E02001694,123
3,7616,E02001694,126
4,21974,E02001694,128
...,...,...,...
7995,10735,E02005384,587
7996,8030,E02005384,640
7997,13361,E02005384,641
7998,21220,E02005384,663


concat three dataframe

In [118]:
output0_10000 = pd.concat([output0, output00, output000], axis=0, ignore_index=True)
output0_10000

,Distance in meter,origin_id,destination_id
0,7681,E02002536,122
1,14506,E02002536,127
2,14764,E02002536,137
3,12744,E02002536,836
4,12605,E02002536,838
...,...,...,...
9995,10735,E02005384,587
9996,8030,E02005384,640
9997,13361,E02005384,641
9998,21220,E02005384,663


In [123]:
output0_10000.to_csv('distance0_10000.csv',index=False)

### 10000-40005:

In [139]:
euclidean_distance_matrix_sub0000

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
10000,11031.204730,E02005383,88,-1.191335,52.604219,-1.344240,52.570149
10001,15420.215926,E02005383,94,-1.135566,52.625556,-1.344240,52.570149
10002,15420.215926,E02005383,95,-1.135566,52.625556,-1.344240,52.570149
10003,14930.542416,E02005383,569,-1.152096,52.635943,-1.344240,52.570149
10004,13677.542416,E02005383,570,-1.145196,52.590552,-1.344240,52.570149
...,...,...,...,...,...,...,...
40000,4650.274235,E02006232,1421,0.717603,52.078173,0.775123,52.056043
40001,15001.038129,E02006233,328,0.951969,52.042708,0.733357,52.042477
40002,1684.007428,E02006233,382,0.752463,52.032980,0.733357,52.042477
40003,4116.444139,E02006233,1421,0.717603,52.078173,0.733357,52.042477


In [137]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

In [140]:
for (i1, row1) in euclidean_distance_matrix_sub0000.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    #result_time = result["rows"][0]["elements"][0]["duration"]["value"]
    #time_list.append(round(result_time/60,1))
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)
    
output0000 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
#output['duration in minutes'] = time_list
output0000['origin_id'] = origin_id_list
output0000['destination_id'] = destination_id_list

In [141]:
output0000

,Distance in meter,origin_id,destination_id
0,18319,E02005383,88
1,20306,E02005383,94
2,20306,E02005383,95
3,18402,E02005383,569
4,20039,E02005383,570
...,...,...,...
30000,7706,E02006232,1421
30001,19250,E02006233,328
30002,2545,E02006233,382
30003,4914,E02006233,1421


In [142]:
output0_40005 = pd.concat([output0_10000, output0000], axis=0, ignore_index=True)
output0_40005

,Distance in meter,origin_id,destination_id
0,7681,E02002536,122
1,14506,E02002536,127
2,14764,E02002536,137
3,12744,E02002536,836
4,12605,E02002536,838
...,...,...,...
40000,7706,E02006232,1421
40001,19250,E02006233,328
40002,2545,E02006233,382
40003,4914,E02006233,1421


In [143]:
output0_40005.to_csv('distance0_40005.csv',index=False)

### 40005-100000:

In [72]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

In [73]:
euclidean_distance_matrix_sub00000

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
40005,9065.184528,E02006230,320,1.026239,51.977340,0.958806,52.047371
40006,699.587089,E02006230,328,0.951969,52.042708,0.958806,52.047371
40007,14250.236081,E02006230,382,0.752463,52.032980,0.958806,52.047371
40008,15882.798416,E02006230,1520,1.188752,52.031249,0.958806,52.047371
40009,3974.639134,E02006231,319,1.148655,52.064023,1.106164,52.039749
...,...,...,...,...,...,...,...
99995,5833.769441,E02000870,848,-0.132245,51.521891,-0.048735,51.527933
99996,4423.187249,E02000870,850,-0.082066,51.561831,-0.048735,51.527933
99997,9593.039885,E02000870,853,-0.185497,51.515375,-0.048735,51.527933
99998,7413.892529,E02000870,854,-0.140230,51.493547,-0.048735,51.527933


In [74]:
for (i1, row1) in euclidean_distance_matrix_sub00000.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    #result_time = result["rows"][0]["elements"][0]["duration"]["value"]
    #time_list.append(round(result_time/60,1))
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

In [75]:
output00000 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
#output['duration in minutes'] = time_list
output00000['origin_id'] = origin_id_list
output00000['destination_id'] = destination_id_list

In [76]:
output00000

,Distance in meter,origin_id,destination_id
0,10712,E02006230,320
1,929,E02006230,328
2,17991,E02006230,382
3,19466,E02006230,1520
4,4820,E02006231,319
...,...,...,...
59990,7497,E02000870,848
59991,5804,E02000870,850
59992,11247,E02000870,853
59993,10704,E02000870,854


### Concat 

In [77]:
output0_40005 = pd.read_csv('distance0_40005.csv')

In [79]:
output0_100000 = pd.concat([output0_40005, output00000], axis=0, ignore_index=True)
output0_100000

,Distance in meter,origin_id,destination_id
0,7681,E02002536,122
1,14506,E02002536,127
2,14764,E02002536,137
3,12744,E02002536,836
4,12605,E02002536,838
...,...,...,...
99995,7497,E02000870,848
99996,5804,E02000870,850
99997,11247,E02000870,853
99998,10704,E02000870,854


### Export

In [80]:
output0_100000.to_csv('distance0_100000.csv',index=False)

## Sub 100000-200000:

In [81]:
euclidean_distance_matrix_sub11=euclidean_distance_matrix[100000:150000]
euclidean_distance_matrix_sub12=euclidean_distance_matrix[150000:200000]
print(euclidean_distance_matrix_sub11.shape,euclidean_distance_matrix_sub12.shape)

(50000, 7) (50000, 7)


In [82]:
euclidean_distance_matrix_sub11

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
100000,12721.386618,E02000870,860,0.043597,51.626764,-0.048735,51.527933
100001,3677.333605,E02000870,862,-0.072245,51.498309,-0.048735,51.527933
100002,2103.234396,E02000870,865,-0.067125,51.542964,-0.048735,51.527933
100003,4691.752632,E02000870,868,-0.115340,51.520680,-0.048735,51.527933
100004,11517.558269,E02000870,869,-0.212988,51.513104,-0.048735,51.527933
...,...,...,...,...,...,...,...
149995,7829.559538,E02000389,1035,-0.196329,51.549901,-0.221199,51.481247
149996,500.182648,E02000389,1040,-0.219751,51.485652,-0.221199,51.481247
149997,6790.272337,E02000389,1042,-0.185118,51.537986,-0.221199,51.481247
149998,6790.272337,E02000389,1043,-0.185118,51.537986,-0.221199,51.481247


In [83]:
euclidean_distance_matrix_sub12

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
150000,7229.077381,E02000389,1051,-0.128218,51.510485,-0.221199,51.481247
150001,13769.297140,E02000389,1054,-0.302779,51.368457,-0.221199,51.481247
150002,2308.794373,E02000389,1057,-0.242890,51.496977,-0.221199,51.481247
150003,12952.131616,E02000389,1058,-0.170637,51.593332,-0.221199,51.481247
150004,3926.097848,E02000389,1060,-0.166139,51.473274,-0.221199,51.481247
...,...,...,...,...,...,...,...
199995,11907.401526,E02000396,1243,-0.049023,51.527756,-0.192873,51.469475
199996,6180.607334,E02000396,1244,-0.127758,51.507351,-0.192873,51.469475
199997,9643.835633,E02000396,1246,-0.089835,51.527611,-0.192873,51.469475
199998,7084.907311,E02000396,1247,-0.171280,51.531724,-0.192873,51.469475


### 100000-150000

In [84]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

In [85]:
for (i1, row1) in euclidean_distance_matrix_sub11.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

In [86]:
output11 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output11['origin_id'] = origin_id_list
output11['destination_id'] = destination_id_list

In [87]:
output11

,Distance in meter,origin_id,destination_id
0,15730,E02000870,860
1,5377,E02000870,862
2,2935,E02000870,865
3,5635,E02000870,868
4,13245,E02000870,869
...,...,...,...
49995,10886,E02000389,1035
49996,1471,E02000389,1040
49997,11488,E02000389,1042
49998,11488,E02000389,1043


### 150000-200000

In [88]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

In [89]:
for (i1, row1) in euclidean_distance_matrix_sub12.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

In [90]:
output12 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output12['origin_id'] = origin_id_list
output12['destination_id'] = destination_id_list

In [91]:
output12

,Distance in meter,origin_id,destination_id
0,7966,E02000389,1051
1,17794,E02000389,1054
2,3432,E02000389,1057
3,24200,E02000389,1058
4,5890,E02000389,1060
...,...,...,...
49995,15244,E02000396,1243
49996,8229,E02000396,1244
49997,12545,E02000396,1246
49998,12622,E02000396,1247


### Concat

In [92]:
output100000_200000 = pd.concat([output11, output12], axis=0, ignore_index=True)
output100000_200000

,Distance in meter,origin_id,destination_id
0,15730,E02000870,860
1,5377,E02000870,862
2,2935,E02000870,865
3,5635,E02000870,868
4,13245,E02000870,869
...,...,...,...
99995,15244,E02000396,1243
99996,8229,E02000396,1244
99997,12545,E02000396,1246
99998,12622,E02000396,1247


### Export

In [93]:
output100000_200000.to_csv('distance100000_200000.csv',index=False)

## Sub 200000-300000:

In [12]:
euclidean_distance_matrix_sub31=euclidean_distance_matrix[200000:250000]
euclidean_distance_matrix_sub32=euclidean_distance_matrix[250000:300000]
print(euclidean_distance_matrix_sub31.shape,euclidean_distance_matrix_sub32.shape)

(50000, 7) (50000, 7)


In [ ]:
euclidean_distance_matrix_sub31

In [17]:
euclidean_distance_matrix_sub32

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
250000,12297.832983,E02000139,28,-0.058869,51.517871,-0.044655,51.407677
250001,3182.738594,E02000139,29,-0.027529,51.381147,-0.044655,51.407677
250002,13060.529690,E02000139,31,-0.042144,51.525073,-0.044655,51.407677
250003,13912.937414,E02000139,32,0.035218,51.522364,-0.044655,51.407677
250004,12808.353111,E02000139,33,-0.100046,51.517496,-0.044655,51.407677
...,...,...,...,...,...,...,...
299995,12739.877296,E02000659,783,-0.093861,51.568424,-0.007815,51.467262
299996,8438.739912,E02000659,784,-0.125915,51.485031,-0.007815,51.467262
299997,10233.755708,E02000659,787,-0.132677,51.516127,-0.007815,51.467262
299998,6051.253126,E02000659,788,-0.032457,51.519438,-0.007815,51.467262


### 200000-250000

In [13]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

In [14]:
for (i1, row1) in euclidean_distance_matrix_sub31.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

In [14]:
output31 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output31['origin_id'] = origin_id_list
output31['destination_id'] = destination_id_list

In [15]:
output31

,Distance in meter,origin_id,destination_id
0,12627,E02000396,1256
1,13555,E02000396,1257
2,3462,E02000396,1259
3,9402,E02000396,1261
4,10529,E02000396,1262
...,...,...,...
49995,10367,E02000139,3
49996,18866,E02000139,8
49997,19417,E02000139,9
49998,16960,E02000139,21


### 250000-296941

In [18]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

In [ ]:
for (i1, row1) in euclidean_distance_matrix_sub32.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

In [21]:
output32 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output32['origin_id'] = origin_id_list
output32['destination_id'] = destination_id_list

### The rest 296941-300000

In [35]:
a = euclidean_distance_matrix_sub32.reset_index(drop=True)
euclidean_distance_matrix_sub33 = a[49641:]
euclidean_distance_matrix_sub33

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
49641,7058.328679,E02000658,1536,-0.109181,51.522312,-0.047235,51.472003
49642,7021.315238,E02000658,1538,-0.106859,51.522982,-0.047235,51.472003
49643,7490.799537,E02000658,1539,-0.134737,51.511383,-0.047235,51.472003
49644,15868.362536,E02000658,1541,-0.264927,51.429008,-0.047235,51.472003
49645,5488.272299,E02000658,1543,-0.098542,51.509529,-0.047235,51.472003
...,...,...,...,...,...,...,...
49995,12739.877296,E02000659,783,-0.093861,51.568424,-0.007815,51.467262
49996,8438.739912,E02000659,784,-0.125915,51.485031,-0.007815,51.467262
49997,10233.755708,E02000659,787,-0.132677,51.516127,-0.007815,51.467262
49998,6051.253126,E02000659,788,-0.032457,51.519438,-0.007815,51.467262


In [41]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

In [42]:
for (i1, row1) in euclidean_distance_matrix_sub33.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

In [43]:
output33 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output33['origin_id'] = origin_id_list
output33['destination_id'] = destination_id_list

In [39]:
output31

,Distance in meter,origin_id,destination_id
0,12627,E02000396,1256
1,13555,E02000396,1257
2,3462,E02000396,1259
3,9402,E02000396,1261
4,10529,E02000396,1262
...,...,...,...
49995,10367,E02000139,3
49996,18866,E02000139,8
49997,19417,E02000139,9
49998,16960,E02000139,21


In [40]:
output32

,Distance in meter,origin_id,destination_id
0,16584,E02000139,28
1,4842,E02000139,29
2,21564,E02000139,31
3,21083,E02000139,32
4,16309,E02000139,33
...,...,...,...
49636,7896,E02000658,1528
49637,7081,E02000658,1529
49638,10553,E02000658,1531
49639,11071,E02000658,1533


In [44]:
output33

,Distance in meter,origin_id,destination_id
0,8511,E02000658,1536
1,8437,E02000658,1538
2,9476,E02000658,1539
3,19796,E02000658,1541
4,6949,E02000658,1543
...,...,...,...
354,19627,E02000659,783
355,9905,E02000659,784
356,12923,E02000659,787
357,10710,E02000659,788


### Concat 

In [46]:
output200000_300000 = pd.concat([output31, output32,output33], axis=0, ignore_index=True)
output200000_300000

,Distance in meter,origin_id,destination_id
0,12627,E02000396,1256
1,13555,E02000396,1257
2,3462,E02000396,1259
3,9402,E02000396,1261
4,10529,E02000396,1262
...,...,...,...
99995,19627,E02000659,783
99996,9905,E02000659,784
99997,12923,E02000659,787
99998,10710,E02000659,788


### Export

In [47]:
output200000_300000.to_csv('distance200000_300000.csv',index=False)

## Sub 300000-400000:

In [49]:
euclidean_distance_matrix_sub31=euclidean_distance_matrix[300000:350000]
euclidean_distance_matrix_sub32=euclidean_distance_matrix[350000:400000]
print(euclidean_distance_matrix_sub31.shape,euclidean_distance_matrix_sub32.shape)

(50000, 7) (50000, 7)


### 300000-350000

In [51]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

In [53]:
for (i1, row1) in euclidean_distance_matrix_sub31.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

In [53]:
output31 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output31['origin_id'] = origin_id_list
output31['destination_id'] = destination_id_list

In [54]:
output31

,Distance in meter,origin_id,destination_id
0,15840,E02000659,793
1,8803,E02000659,794
2,12695,E02000659,796
3,11853,E02000659,797
4,11877,E02000659,801
...,...,...,...
49995,10063,E02000792,1191
49996,20837,E02000792,1194
49997,20037,E02000792,1195
49998,18622,E02000792,1196


### 350000-400000

In [55]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

In [56]:
for (i1, row1) in euclidean_distance_matrix_sub32.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

In [57]:
output32 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output32['origin_id'] = origin_id_list
output32['destination_id'] = destination_id_list

In [59]:
output31

,Distance in meter,origin_id,destination_id
0,15840,E02000659,793
1,8803,E02000659,794
2,12695,E02000659,796
3,11853,E02000659,797
4,11877,E02000659,801
...,...,...,...
49995,10063,E02000792,1191
49996,20837,E02000792,1194
49997,20037,E02000792,1195
49998,18622,E02000792,1196


### Concat

In [60]:
output300000_400000 = pd.concat([output31, output32], axis=0, ignore_index=True)
output300000_400000

,Distance in meter,origin_id,destination_id
0,15840,E02000659,793
1,8803,E02000659,794
2,12695,E02000659,796
3,11853,E02000659,797
4,11877,E02000659,801
...,...,...,...
99995,28523,E02000168,1416
99996,9903,E02000168,1417
99997,19144,E02000168,1418
99998,6532,E02000168,1419


### Export

In [61]:
output300000_400000.to_csv('distance300000_400000.csv',index=False)

## Sub 400000-500000:

In [5]:
euclidean_distance_matrix_sub41=euclidean_distance_matrix[400000:450000]
euclidean_distance_matrix_sub42=euclidean_distance_matrix[450000:500000]
print(euclidean_distance_matrix_sub41.shape,euclidean_distance_matrix_sub42.shape)

(50000, 7) (50000, 7)


In [8]:
euclidean_distance_matrix_sub41

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
400000,12900.464406,E02000168,1427,-0.005899,51.476389,-0.141545,51.555674
400001,4157.761144,E02000168,1429,-0.142052,51.518299,-0.141545,51.555674
400002,16072.178742,E02000168,1430,-0.370928,51.576620,-0.141545,51.555674
400003,2611.759842,E02000168,1435,-0.109058,51.567558,-0.141545,51.555674
400004,6613.840377,E02000168,1439,-0.147268,51.496325,-0.141545,51.555674
...,...,...,...,...,...,...,...
449995,9336.127719,E02000680,1319,-0.127758,51.507351,-0.061117,51.434470
449996,9387.397639,E02000680,1322,-0.123868,51.509207,-0.061117,51.434470
449997,10936.790690,E02000680,1325,-0.122476,51.525014,-0.061117,51.434470
449998,8918.074515,E02000680,1331,-0.187495,51.420838,-0.061117,51.434470


In [7]:
euclidean_distance_matrix_sub42

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
450000,3396.824777,E02000680,1335,-0.012818,51.429889,-0.061117,51.434470
450001,15263.192338,E02000680,1337,-0.166238,51.554972,-0.061117,51.434470
450002,13846.557259,E02000680,1341,-0.090231,51.557610,-0.061117,51.434470
450003,4683.921898,E02000680,1343,-0.128472,51.435232,-0.061117,51.434470
450004,11551.855010,E02000680,1349,0.104420,51.425747,-0.061117,51.434470
...,...,...,...,...,...,...,...
499995,8492.718419,E02000694,262,-0.185719,51.496029,-0.188459,51.419701
499996,10623.230832,E02000694,269,-0.127758,51.507351,-0.188459,51.419701
499997,12936.577145,E02000694,281,-0.108915,51.524852,-0.188459,51.419701
499998,4458.388885,E02000694,286,-0.127843,51.432754,-0.188459,51.419701


### 400000-450000

In [9]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

In [10]:
for (i1, row1) in euclidean_distance_matrix_sub41.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

In [11]:
output41 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output41['origin_id'] = origin_id_list
output41['destination_id'] = destination_id_list

In [12]:
output41

,Distance in meter,origin_id,destination_id
0,16898,E02000168,1427
1,4809,E02000168,1429
2,27703,E02000168,1430
3,3523,E02000168,1435
4,8982,E02000168,1439
...,...,...,...
49995,11655,E02000680,1319
49996,11624,E02000680,1322
49997,13187,E02000680,1325
49998,13186,E02000680,1331


### 450000-500000

In [13]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

In [14]:
for (i1, row1) in euclidean_distance_matrix_sub42.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

In [15]:
output42 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output42['origin_id'] = origin_id_list
output42['destination_id'] = destination_id_list

In [16]:
output42

,Distance in meter,origin_id,destination_id
0,4637,E02000680,1335
1,18154,E02000680,1337
2,16909,E02000680,1341
3,7029,E02000680,1343
4,14817,E02000680,1349
...,...,...,...
49995,10712,E02000694,262
49996,14152,E02000694,269
49997,15559,E02000694,281
49998,6199,E02000694,286


### Concat 

In [17]:
output400000_500000 = pd.concat([output41, output42], axis=0, ignore_index=True)
output400000_500000

,Distance in meter,origin_id,destination_id
0,16898,E02000168,1427
1,4809,E02000168,1429
2,27703,E02000168,1430
3,3523,E02000168,1435
4,8982,E02000168,1439
...,...,...,...
99995,10712,E02000694,262
99996,14152,E02000694,269
99997,15559,E02000694,281
99998,6199,E02000694,286


### Export

In [18]:
output400000_500000.to_csv('distance400000_500000.csv',index=False)

## Sub 500000-555158:

In [97]:
euclidean_distance_matrix_sub5=euclidean_distance_matrix[500000:555158]
print(euclidean_distance_matrix_sub5.shape)

(55158, 7)


In [98]:
euclidean_distance_matrix_sub5

,euclidean,origin,dest,site_lon,site_lat,MSOA_lon,MSOA_lat
500000,10465.297974,E02000694,289,-0.042311,51.442175,-0.188459,51.419701
500001,13491.387581,E02000694,291,-0.096436,51.526504,-0.188459,51.419701
500002,11624.353059,E02000694,293,-0.176444,51.523931,-0.188459,51.419701
500003,11537.108597,E02000694,294,-0.088010,51.502280,-0.188459,51.419701
500004,10623.230832,E02000694,310,-0.127758,51.507351,-0.188459,51.419701
...,...,...,...,...,...,...,...
555153,12599.535851,E02003088,1204,-2.754862,51.433531,-2.815479,51.326784
555154,5650.422238,E02006070,1278,-3.015693,51.082446,-2.977239,51.127113
555155,14527.209707,E02006679,1176,-2.345549,51.360932,-2.136956,51.364067
555156,9895.394094,E02006679,1177,-1.996076,51.352366,-2.136956,51.364067


In [99]:
API_key = '' #enter your google maps api key here
gmaps = googlemaps.Client(key=API_key)

#empty list - will be used to store calculated distances
#time_list = []
distance_list = []
origin_id_list = []
destination_id_list = []

In [100]:
for (i1, row1) in euclidean_distance_matrix_sub5.iterrows():
    LatOrigin = row1['MSOA_lat']
    LongOrigin = row1['MSOA_lon']
    origin = (LatOrigin, LongOrigin)
    origin_id = row1['origin']
    
    LatDestination = row1['site_lat']
    LongDestination = row1['site_lon'] 
    destination_id = row1['dest']
    destination = (LatDestination, LongDestination)
    
    result = gmaps.distance_matrix(origin, destination, mode='driving')
    result_distance = result["rows"][0]["elements"][0]["distance"]["value"]
    distance_list.append(result_distance)
    origin_id_list.append(origin_id)
    destination_id_list.append(destination_id)

In [101]:
output5 = pd.DataFrame(distance_list, columns = ['Distance in meter'])
output5['origin_id'] = origin_id_list
output5['destination_id'] = destination_id_list

In [102]:
output5

,Distance in meter,origin_id,destination_id
0,14292,E02000694,289
1,16752,E02000694,291
2,15372,E02000694,293
3,12966,E02000694,294
4,14152,E02000694,310
...,...,...,...
55153,15948,E02003088,1204
55154,7966,E02006070,1278
55155,21577,E02006679,1176
55156,10890,E02006679,1177


### Export

In [103]:
output5.to_csv('distance500000_555158.csv',index=False)